# Start

In [2]:
x=1

In [5]:
from insightface.app import FaceAnalysis
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640,640))  # -1 = CPU, 0 = first GPU
print("InsightFace ready.")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0', 'fuse_conv_bias': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\Parvez/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider

In [6]:
# enroll_face.py
import argparse, json, cv2, numpy as np
from pathlib import Path
from insightface.app import FaceAnalysis

def _pick_ctx_id(device: str, gpu_id: int) -> int:
    """
    Returns  GPU index if CUDA provider is available and device=='gpu', else -1 (CPU).
    """
    if device.lower() != "gpu":
        return -1
    try:
        import onnxruntime as ort
        providers = ort.get_available_providers()
        if "CUDAExecutionProvider" in providers:
            return int(gpu_id)
        else:
            print("[WARN] CUDAExecutionProvider not available; falling back to CPU.")
            return -1
    except Exception as e:
        print(f"[WARN] onnxruntime check failed ({e}); falling back to CPU.")
        return -1

def enroll(out_dir="whitelist", cam=0, shots=20, device="gpu", gpu_id=0, det_size=640):
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # Prepare InsightFace on GPU if available, else CPU
    ctx_id = _pick_ctx_id(device, gpu_id)
    print(f"[Enroll] Initializing FaceAnalysis (ctx_id={ctx_id})...")
    app = FaceAnalysis(name="buffalo_l")
    app.prepare(ctx_id=ctx_id, det_size=(det_size, det_size))
    print("[Enroll] InsightFace ready.")

    cap = cv2.VideoCapture(cam)
    if not cap.isOpened():
        print(f"[ERROR] Cannot open camera source: {cam}")
        return

    embs = []
    print("[Enroll] Look at the camera; press 'q' to stop early.")
    while len(embs) < shots:
        ok, frame = cap.read()
        if not ok:
            print("[WARN] Frame grab failed; stopping.")
            break

        faces = app.get(frame)
        if faces:
            # Take the largest face
            f = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
            if getattr(f, "normed_embedding", None) is not None:
                embs.append(f.normed_embedding.astype(float))
            # Visual feedback
            x1, y1, x2, y2 = map(int, f.bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

        cv2.imshow("Enroll", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release(); cv2.destroyAllWindows()

    if not embs:
        print("[Enroll] No face embeddings captured.")
        return

    mean_emb = np.mean(np.stack(embs, axis=0), axis=0)
    out = Path(out_dir) / "creator_embedding.json"
    out.write_text(json.dumps({"embedding": mean_emb.tolist()}), encoding="utf-8")
    print(f"[Enroll] Saved: {out}")

if __name__ == "__main__":
    ap = argparse.ArgumentParser(description="Enroll the creator's face embedding (GPU-first).")
    ap.add_argument("--out_dir", default="whitelist")
    ap.add_argument("--cam", type=int, default=0)
    ap.add_argument("--shots", type=int, default=20)
    ap.add_argument("--device", choices=["gpu","cpu"], default="gpu")
    ap.add_argument("--gpu_id", type=int, default=0, help="GPU index (0 for first GPU)")
    ap.add_argument("--det_size", type=int, default=640, help="Detector input size (square)")
    args = ap.parse_args()

    enroll(out_dir=args.out_dir, cam=args.cam, shots=args.shots,
           device=args.device, gpu_id=args.gpu_id, det_size=args.det_size)


usage: ipykernel_launcher.py [-h] [--out_dir OUT_DIR] [--cam CAM]
                             [--shots SHOTS] [--device {gpu,cpu}]
                             [--gpu_id GPU_ID] [--det_size DET_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\Parvez\AppData\Roaming\jupyter\runtime\kernel-v34a6edcfbf3b113f3abc980e7e65ea07bb0594ee8.json


SystemExit: 2

c:\Computer Science\Projects\tiktok-techjam-2025\venv_notebook\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Blur the faces